In [3]:
# general imports
import sys
import os.path
import warnings
import time

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import torchvision.datasets as datasets

import numpy as np
from RerF import fastRerF, fastPredict
from multiprocessing import cpu_count
from sklearn.metrics import accuracy_score

sns.set()
warnings.filterwarnings("ignore")
# sys.stdout = open("deep_conv_rf_logs.txt", "w+")

In [4]:
##########
# Settings
##########
# base_path = ""
base_path = "rerf/3vs5/"

class_one = 3
class_two = 5

fraction_of_train_samples_space = np.geomspace(0.01, 0.35, num=10)

In [5]:
###########################################################################################################
# Data Preparation
###########################################################################################################

def normalize(x):
    scale = np.mean(np.arange(0, 256))
    return (x - scale) / scale


# train data
cifar_trainset = datasets.CIFAR10(root='../../data', train=True, download=True, transform=None)
cifar_train_images = normalize(cifar_trainset.data)
cifar_train_labels = np.array(cifar_trainset.targets)

# test data
cifar_testset = datasets.CIFAR10(root='../../data', train=False, download=True, transform=None)
cifar_test_images = normalize(cifar_testset.data)
cifar_test_labels = np.array(cifar_testset.targets)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
class ConvRF(object):
    def __init__(self, kernel_size=5, stride=2):
        self.kernel_size = kernel_size
        self.stride = stride
        self.kernel_forest = None
        self.num_trees = 1000

    def _convolve_chop(self, images, labels=None, flatten=False):

        batch_size, in_dim, _, num_channels = images.shape

        out_dim = int((in_dim - self.kernel_size) / self.stride) + 1  # calculate output dimensions

        # create matrix to hold the chopped images
        out_images = np.zeros((batch_size, out_dim, out_dim,
                               self.kernel_size, self.kernel_size, num_channels))
        out_labels = None

        curr_y = out_y = 0
        # move kernel vertically across the image
        while curr_y + self.kernel_size <= in_dim:
            curr_x = out_x = 0
            # move kernel horizontally across the image
            while curr_x + self.kernel_size <= in_dim:
                # chop images
                out_images[:, out_x, out_y] = images[:, curr_x:curr_x +
                                                     self.kernel_size, curr_y:curr_y+self.kernel_size, :]
                curr_x += self.stride
                out_x += 1
            curr_y += self.stride
            out_y += 1

        if flatten:
            out_images = out_images.reshape(batch_size, out_dim, out_dim, -1)

        if labels is not None:
            out_labels = np.zeros((batch_size, out_dim, out_dim))
            out_labels[:, ] = labels.reshape(-1, 1, 1)

        return out_images, out_labels

    def convolve_fit(self, images, labels):
        sub_images, sub_labels = self._convolve_chop(images, labels=labels, flatten=True)

        batch_size, out_dim, _, _ = sub_images.shape

        all_sub_images = sub_images.reshape(batch_size*out_dim*out_dim, -1)
        all_sub_labels = sub_labels.reshape(batch_size*out_dim*out_dim, -1)

        self.kernel_forest = fastRerF(X=all_sub_images,
                                      Y=all_sub_labels,
                                      forestType="binnedBase",
                                      trees=self.num_trees,
                                      numCores=cpu_count() - 1)

        convolved_image = np.zeros((images.shape[0], out_dim, out_dim, 1))
        for i in range(out_dim):
            for j in range(out_dim):
                data = sub_images[:, i, j].flatten().tolist()
                convolved_image[:, i, j] = np.array(self.kernel_forest.predict_post(data)[1] / float(self.num_trees))[..., np.newaxis]
        return convolved_image

    def convolve_predict(self, images):
        if not self.kernel_forest:
            raise Exception("Should fit training data before predicting")

        sub_images, _ = self._convolve_chop(images, flatten=True)

        batch_size, out_dim, _, _ = sub_images.shape

        kernel_predictions = np.zeros((images.shape[0], out_dim, out_dim, 1))

        for i in range(out_dim):
            for j in range(out_dim):
                data = sub_images[:, i, j].flatten().tolist()
                kernel_predictions[:, i, j] = np.array(self.kernel_forest.predict_post(data)[1] / float(self.num_trees))[..., np.newaxis]
        return kernel_predictions

In [7]:
def run_naive_rf(train_images, train_labels, test_images, test_labels, fraction_of_train_samples, class1=3, class2=5):
    num_train_samples_class_1 = int(np.sum(train_labels == class1) * fraction_of_train_samples)
    num_train_samples_class_2 = int(np.sum(train_labels == class2) * fraction_of_train_samples)

    # get only train images and labels for class 1 and class 2
    train_images = np.concatenate([train_images[train_labels == class1][:num_train_samples_class_1],
                                   train_images[train_labels == class2][:num_train_samples_class_2]])
    train_labels = np.concatenate(
        [np.repeat(0, num_train_samples_class_1), np.repeat(1, num_train_samples_class_2)])

    # get only test images and labels for class 1 and class 2
    test_images = np.concatenate([test_images[test_labels == class1],
                                  test_images[test_labels == class2]])
    test_labels = np.concatenate(
        [np.repeat(0, np.sum(test_labels == class1)), np.repeat(1, np.sum(test_labels == class2))])

    # Train
    forest = fastRerF(X=train_images.reshape(-1, 32*32*3),
                      Y=train_labels,
                      forestType="binnedBase",
                      trees=100,
                      numCores=cpu_count() - 1)
    # forest.printParameters()

    # Test
    test_preds = fastPredict(test_images.reshape(-1, 32*32*3), forest)
    return accuracy_score(test_labels, test_preds)

In [8]:
def run_one_layer_deep_conv_rf(train_images, train_labels, test_images, test_labels, fraction_of_train_samples, class1=3, class2=5):
    num_train_samples_class_1 = int(np.sum(train_labels == class1) * fraction_of_train_samples)
    num_train_samples_class_2 = int(np.sum(train_labels == class2) * fraction_of_train_samples)

    # get only train images and labels for class 1 and class 2
    train_images = np.concatenate([train_images[train_labels == class1][:num_train_samples_class_1],
                                   train_images[train_labels == class2][:num_train_samples_class_2]])
    train_labels = np.concatenate(
        [np.repeat(0, num_train_samples_class_1), np.repeat(1, num_train_samples_class_2)])

    # get only test images and labels for class 1 and class 2
    test_images = np.concatenate([test_images[test_labels == class1],
                                  test_images[test_labels == class2]])
    test_labels = np.concatenate(
        [np.repeat(0, np.sum(test_labels == class1)), np.repeat(1, np.sum(test_labels == class2))])

    # Train
    # ConvRF (layer 1)
    conv1 = ConvRF(kernel_size=10, stride=2)
    conv1_map = conv1.convolve_fit(train_images, train_labels)

    # Full RF
    conv1_full_RF = fastRerF(X=conv1_map.reshape(len(train_images), -1),
                             Y=train_labels,
                             forestType="binnedBase",
                             trees=100,
                             numCores=cpu_count() - 1)

    # Test (after ConvRF 1 and Full RF)
    conv1_map_test = conv1.convolve_predict(test_images)
    test_preds = fastPredict(conv1_map_test.reshape(len(test_images), -1), conv1_full_RF)

    return accuracy_score(test_labels, test_preds)

In [9]:
def run_two_layer_deep_conv_rf(train_images, train_labels, test_images, test_labels, fraction_of_train_samples, class1=3, class2=5):
    num_train_samples_class_1 = int(np.sum(train_labels == class1) * fraction_of_train_samples)
    num_train_samples_class_2 = int(np.sum(train_labels == class2) * fraction_of_train_samples)

    # get only train images and labels for class 1 and class 2
    train_images = np.concatenate([train_images[train_labels == class1][:num_train_samples_class_1],
                                   train_images[train_labels == class2][:num_train_samples_class_2]])
    train_labels = np.concatenate(
        [np.repeat(0, num_train_samples_class_1), np.repeat(1, num_train_samples_class_2)])

    # get only test images and labels for class 1 and class 2
    test_images = np.concatenate([test_images[test_labels == class1],
                                  test_images[test_labels == class2]])
    test_labels = np.concatenate(
        [np.repeat(0, np.sum(test_labels == class1)), np.repeat(1, np.sum(test_labels == class2))])

    # Train
    # ConvRF (layer 1)
    conv1 = ConvRF(kernel_size=10, stride=2)
    conv1_map = conv1.convolve_fit(train_images, train_labels)

    # ConvRF (layer 2)
    conv2 = ConvRF(kernel_size=7, stride=1)
    conv2_map = conv2.convolve_fit(conv1_map, train_labels)

    # Full RF
    conv1_full_RF = fastRerF(X=conv2_map.reshape(len(train_images), -1),
                             Y=train_labels,
                             forestType="binnedBase",
                             trees=100,
                             numCores=cpu_count() - 1)

    # Test (after ConvRF 2 and Full RF)
    conv1_map_test = conv1.convolve_predict(test_images)
    conv2_map_test = conv2.convolve_predict(conv1_map_test)
    test_preds = fastPredict(conv2_map_test.reshape(len(test_images), -1), conv1_full_RF)

    return accuracy_score(test_labels, test_preds)

In [10]:
###############################################################################
# Experiments
###############################################################################

def print_old_results(file_name):
    global fraction_of_train_samples_space
    accuracy_scores = np.load(file_name)
    for fraction_of_train_samples, (best_accuracy, time_taken) in zip(fraction_of_train_samples_space, accuracy_scores):
        print("Train Fraction:", str(fraction_of_train_samples))
        print("Accuracy:", str(best_accuracy))
        print("Experiment Runtime: " + str(time_taken), "\n")
        print()
    return accuracy_scores


def run_experiment(experiment, experiment_result_file, text, cnn_model=None, class1=class_one, class2=class_two):
    global fraction_of_train_samples_space
    repeats = 2

    print("##################################################################")
    print("acc vs n_samples: " + text + "\n")
    acc_vs_n = list()
    file_name = base_path+experiment_result_file+".npy"
    if not os.path.exists(file_name):
        for fraction_of_train_samples in fraction_of_train_samples_space:
            if not cnn_model:
                start = time.time()
                best_accuracy = np.mean([experiment(cifar_train_images, cifar_train_labels, cifar_test_images,
                                                    cifar_test_labels, fraction_of_train_samples, class1, class2) for _ in range(repeats)])
                end = time.time()
            else:
                start = time.time()
                best_accuracy = np.mean([experiment(cnn_model, cifar_train_images, cifar_train_labels, cifar_test_images,
                                                    cifar_test_labels, fraction_of_train_samples, class1, class2) for _ in range(repeats)])
                end = time.time()
            time_taken = (end - start)/float(repeats)
            acc_vs_n.append((best_accuracy, time_taken))
            print("Train Fraction:", str(fraction_of_train_samples))
            print("Accuracy:", str(best_accuracy))
            print("Experiment Runtime: " + str(time_taken), "\n")
        np.save(file_name, acc_vs_n)
    else:
        acc_vs_n = print_old_results(file_name)
    print("##################################################################")

    return acc_vs_n

In [11]:
naive_rf_acc_vs_n, naive_rf_acc_vs_n_times = list(zip(*run_experiment(run_naive_rf, "naive_rf_acc_vs_n", "Naive RF")))

##################################################################
acc vs n_samples: Naive RF

Train Fraction: 0.01
Accuracy: 0.50725
Experiment Runtime: 0.34542524814605713 


Train Fraction: 0.014844415983612417
Accuracy: 0.5217499999999999
Experiment Runtime: 0.21671605110168457 


Train Fraction: 0.02203566858945278
Accuracy: 0.52825
Experiment Runtime: 0.27205920219421387 


Train Fraction: 0.03271066310188591
Accuracy: 0.5722499999999999
Experiment Runtime: 0.3542443513870239 


Train Fraction: 0.04855706901841959
Accuracy: 0.573
Experiment Runtime: 0.476399302482605 


Train Fraction: 0.07208013314543994
Accuracy: 0.6105
Experiment Runtime: 0.7783915996551514 


Train Fraction: 0.10699874805650797
Accuracy: 0.60475
Experiment Runtime: 1.068491816520691 


Train Fraction: 0.1588333925876545
Accuracy: 0.61625
Experiment Runtime: 1.6027202606201172 


Train Fraction: 0.23577889516595646
Accuracy: 0.6339999999999999
Experiment Runtime: 2.341895341873169 


Train Fraction: 0.35
Accur

In [ ]:
deep_conv_rf_acc_vs_n, deep_conv_rf_acc_vs_n_times = list(zip(*run_experiment(run_one_layer_deep_conv_rf, "deep_conv_rf_acc_vs_n", "DeepConvRF (shared)")))

##################################################################
acc vs n_samples: DeepConvRF (shared)

Train Fraction: 0.01
Accuracy: 0.5
Experiment Runtime: 10.679657816886902 

Train Fraction: 0.014844415983612417
Accuracy: 0.5
Experiment Runtime: 6.628674149513245 

Train Fraction: 0.02203566858945278
Accuracy: 0.5
Experiment Runtime: 7.467689633369446 

Train Fraction: 0.03271066310188591
Accuracy: 0.5
Experiment Runtime: 8.483198761940002 

Train Fraction: 0.04855706901841959
Accuracy: 0.5
Experiment Runtime: 11.910579562187195 

Train Fraction: 0.07208013314543994
Accuracy: 0.5
Experiment Runtime: 14.807709813117981 

Train Fraction: 0.10699874805650797
Accuracy: 0.5
Experiment Runtime: 19.79778015613556 

Train Fraction: 0.1588333925876545
Accuracy: 0.5
Experiment Runtime: 34.35388112068176 



In [ ]:
deep_conv_rf_two_layer_acc_vs_n, deep_conv_rf_two_layer_acc_vs_n_times = list(zip(*run_experiment(run_two_layer_deep_conv_rf, "deep_conv_rf_two_layer_acc_vs_n", "DeepConvRF (2-layer, shared)")))

##################################################################
acc vs n_samples: DeepConvRF (2-layer, shared)

Train Fraction: 0.01
Accuracy: 0.5
Experiment Runtime: 10.253974914550781 

